In [1]:
"""
ECG 5000 problem (Table 3)
"""

'\nECG 5000 problem (Table 3)\n'

In [2]:
#Import packages to facilitate this research
import os, random
SEED = 1                               
os.environ["PYTHONHASHSEED"]  = str(SEED) 
os.environ["TF_DETERMINISTIC_OPS"] = "1" 
random.seed(SEED)
import numpy as np 
np.random.seed(SEED) 
import tensorflow as tf #(https://www.tensorflow.org/install)
tf.keras.utils.set_random_seed(SEED)      
tf.config.experimental.enable_op_determinism() 
from tensorflow.keras import (layers,Model) 
from tensorflow.keras.layers import (Layer, Input, Dense, Concatenate, Activation, TimeDistributed,
Flatten, Lambda, GlobalAveragePooling1D, LeakyReLU) 
from tensorflow.keras.utils import to_categorical 
from concretedropout.tensorflow import (ConcreteDenseDropout, ConcreteSpatialDropout1D, # https://github.com/aurelio-amerio/ConcreteDropout
get_weight_regularizer, get_dropout_regularizer) 
from sktime.classification.deep_learning import InceptionTimeClassifier #https://www.sktime.net/en/stable/get_started.html
from sktime.classification.ensemble import BaggingClassifier
from aeon.classification.convolution_based import MultiRocketHydraClassifier #https://www.aeon-toolkit.org/en/stable/installation.html
from aeon.classification.early_classification import TEASER
from aeon.classification.hybrid import HIVECOTEV2
from aeon.datasets import load_classification 
from sklearn.metrics import (accuracy_score, mean_absolute_error, mean_squared_error,  # https://scikit-learn.org/stable/install.html
classification_report)
from sklearn.model_selection import train_test_split
from math import comb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from statsmodels.stats.contingency_tables import mcnemar

/opt/anaconda3/lib/python3.12/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [3]:
X, y = load_classification("ECG5000")  #Load data set 
X = X.reshape(X.shape[0], -1) 
y = y.astype(int) 
y = y-1  #Start the classes from 0 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Generate a train-test split of the data 
def create_forecast_data(data, n_forecast): 
    """
    Function that takes as input time series and outputs the observed realizations of the time series. 
    """
    X_forecast = []
    for series in data:
        if len(series) > n_forecast:
            X_forecast.append(series[:-n_forecast])  
    return np.array(X_forecast)  
n_forecast = 5 #The number of steps ahead to forecast
X_observed_train = create_forecast_data(X_train, n_forecast) #Remove the future observations from the observed time series training set
X_observed_train = X_observed_train.astype(np.float32) 
X_forecast_train = np.array([series[-n_forecast:] for series in X_train if len(series) > n_forecast]) #Create the future observations training set
X_forecast_train = X_forecast_train.astype(np.float32)
X_observed_test  = create_forecast_data(X_test, n_forecast) #Remove the future observations from the observed time series test set 
X_observed_test = X_observed_test.astype(np.float32)  
X_forecast_test = np.array([series[-n_forecast:] for series in X_test if len(series)>n_forecast]) #Create the future observations test set
X_forecast_test = X_forecast_test.astype(np.float32)
n_classes = len(np.unique(y_train)) #Number of classes in this problem 
y_train = to_categorical(y_train, num_classes = n_classes) #One hot encode the class labels for the training data 
y_test = to_categorical(y_test, num_classes = n_classes) #One hot encode the class labels for the test data 
X_observed_train = np.expand_dims(X_observed_train, axis = -1) 
X_observed_test = np.expand_dims(X_observed_test, axis = -1) 
X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_observed_train.shape)
print(X_observed_test.shape)
print(X_forecast_train.shape)
print(X_forecast_test.shape)

(4000, 140, 1)
(1000, 140, 1)
(4000, 5)
(1000, 5)
(4000, 135, 1)
(1000, 135, 1)
(4000, 5)
(1000, 5)


In [5]:
#Occasionally Inception can give an error. If this happens rerun the code block again. 
clf = BaggingClassifier(InceptionTimeClassifier(n_epochs=100, batch_size = 64),n_estimators=5,bootstrap=False) #Initialize the Inception time classifier
tic = time.perf_counter() #Time how long it takes for training 
clf.fit(X_observed_train.transpose(0,2,1), np.argmax(y_train,axis = 1)) #Train the Inception time classifier on only the observed time series 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #Prints the time it took the Inception Time classifier to train
clf_predictions1 = clf.predict(X_observed_test.transpose(0,2,1)) #Evaluate the trained Inception time classifier on the test set
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),clf_predictions1)
print(f"Classification Test Accuracy: {classification_accuracy}") #Prints the classification accuracy of the trained Inception time classifier on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),clf_predictions1,digits = 3) #Evaluate the trained Inception time classifier on the test set in terms of classification accuracy and F1 scores 
print(classification__report)

Time taken for fitting: 4507.82 seconds
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step
Classification Test Accuracy: 0.956
              precision    recall  f1-score   support

           0      0.998     0.997     0.997       586
           1      0.931     0.960     0.946       353
           2      0.722     0.722     0.722        18
           3      0.625     0.526     0.571        38
           4      0.000     0.000     0.000         5

    accuracy                          0.956      1000
   macro avg      0.655     0.641     0.647      1000
weighted avg      0.951     0.956     0.953      1000



In [6]:
MRhydra = MultiRocketHydraClassifier() #Initialize the MR-Hydra classifier  
tic = time.perf_counter() #Time how long it takes for training
MRhydra.fit(X_observed_train.transpose(0, 2, 1), np.argmax(y_train,axis = 1))  #Train the MR-Hydra classifier on the observed time series 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #Prints the time it took the MR-Hydra  classifier to train
MRpredictions = MRhydra.predict(X_observed_test.transpose(0, 2, 1)) #Evaluate the trained MR-Hydra classifier on the test set
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), MRpredictions)
print(classification_accuracy) #Prints the classification accuracy of the trained MR-Hydra classifier on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),MRpredictions,digits = 3) #Evaluate the trained MR-Hydra classifier on the test set in terms of classification accuracy and F1 scores
print(classification__report)

Time taken for fitting: 50.47 seconds
0.958
              precision    recall  f1-score   support

           0      0.995     1.000     0.997       586
           1      0.925     0.972     0.948       353
           2      0.706     0.667     0.686        18
           3      0.773     0.447     0.567        38
           4      0.000     0.000     0.000         5

    accuracy                          0.958      1000
   macro avg      0.680     0.617     0.639      1000
weighted avg      0.951     0.958     0.953      1000



In [7]:
early = TEASER(classification_points=[135]) #Initalize the teaser classifier, specifying we want to make the classification decision after we have observed 135 time points 
tic = time.perf_counter() #Time how long it takes for training
early.fit(X_train.transpose(0,2,1), np.argmax(y_train,axis = 1)) #Train the teaser classifier 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #Prints the time it took the TEASER classifier to train
Teaserpredictions, decisions = early.predict(X_test.transpose(0,2,1)) #Evaluate the trained teaser classifier on the test set  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), Teaserpredictions)
print(f"Classification Test Accuracy: {classification_accuracy}") #Prints the classification accuracy of the trained TEASER classifier on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),Teaserpredictions,digits = 3) #Evaluate the trained TEASER classifier on the test set in terms of classification accuracy and F1 scores
print(classification__report)

Time taken for fitting: 389.56 seconds
Classification Test Accuracy: 0.956
              precision    recall  f1-score   support

           0      0.993     0.998     0.996       586
           1      0.918     0.983     0.949       353
           2      0.611     0.611     0.611        18
           3      0.867     0.342     0.491        38
           4      0.000     0.000     0.000         5

    accuracy                          0.956      1000
   macro avg      0.678     0.587     0.609      1000
weighted avg      0.950     0.956     0.948      1000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
Ns = X_observed_train.shape[0] #Number of time series 
time_steps = X_observed_train.shape[1] #Number of time steps per time series 
n_forecast = X_forecast_train.shape[1] #Number of steps ahead for the forecasts 
n_classes = y_train.shape[1] #Number of classes 
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet

In [9]:
def create_custom_filters(filter_length):
    """
    Function that takes in a user specified filter length to then construct hand-crafted filters 
    according to (4)-(6) in Section 5 of the research paper. 
    """
    filter_1 = np.array([(-1) ** (i + 1) for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Decreasing filter
    filter_2 = np.array([(-1) ** i for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Increasing filter
    filter_3 = np.zeros(filter_length) #Peaks filter
    for n in range(0, int(((filter_length - 3) / 3) + 1)): 
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), n)
    for n in range(int(filter_length / 3), int((2 * (filter_length - 3) / 3) + 2)):
        filter_3[n] = (6 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (filter_length / 3)))
    for n in range(int((2 * (filter_length - 3) / 3) + 2), filter_length):
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (2 * (filter_length - 3) / 3) - 2))
    filter_3 = filter_3.reshape(filter_length, 1, 1)
    custom_filters = np.concatenate([filter_1, filter_2, filter_3], axis=2) #Combine the filters together
    return custom_filters
class WelfordMeanVarianceLayer(Layer): #Defining the Welford Mean-variance layer (section 4.2) as a subclass of Layer in Keras
    def __init__(self, total_series, n_forecast=4, **kwargs): #Constructor of the class, where it takes in the number of time series we want to have mean and variance estimates for and the number of forecasts for each 
        super(WelfordMeanVarianceLayer, self).__init__(**kwargs) #Calls the Keras layer class
        self.total_series = total_series #Number of series the layer will track
        self.n_forecast = n_forecast #Number of forecast steps ahead
        self.stats = self.add_weight( #Creates a non-trainable tensor to store the means and variances which is initialized with zeros
            shape=(self.total_series, self.n_forecast, 2),
            initializer="zeros",
            trainable=False,
            name="stats"
        ) 
        self.count = self.add_weight( #Creates a non-trainable tensor to count how many samples for the mean and variance we have for the forecasts of a given time series, which are obtained through successive forward passes
            shape=(self.total_series,),
            initializer="zeros",
            trainable=False,
            name="count"
        )
    def reset_statistics(self): #A method to clear the existing estimates of the mean and variance of the forecasts and set them to zero, which you might want to do during testing
        self.stats.assign(tf.zeros_like(self.stats))
        self.count.assign(tf.zeros_like(self.count))
    def call(self, inputs, **kwargs): #Main method used when calling the Welford layer in a given forward pass
        forecast_values, series_ids = inputs  #Unpacks the inputs into the Welford layer, the forecast values in the current forward pass and the ids of the current time series being processed 
        batch_size = tf.shape(forecast_values)[0] #Derives the current batch size from the shape of the forecast values tensor
        def loop_body(i, current_stats, current_count): #Defines the body of a while loop to be used in the proceeding lines. Processes the ith time series in the batch by applying Welford's algorithm 
            idx = series_ids[i]   #Determines the id of the current time series being processed    
            val = forecast_values[i] #The values to be used to update the means and variances of the forecasts
            current_count = tf.tensor_scatter_nd_add(current_count, [[idx]], [1.0]) #Increments the count of the number of times we have updated the mean and variance of the given time series 
            old_mean = current_stats[idx, :, 0] #Extracts the old forecast means
            old_var  = current_stats[idx, :, 1] #Extracts the old forecast variance 
            n = current_count[idx] #Number of times we have updated the mean and variance of the forecasts for a given time series
            delta = val - old_mean #The difference between the new forecast value and the old mean 
            new_mean = old_mean + (delta /n) #Formula used in Welford's algorithm to update the mean 
            delta2 = val - new_mean #Difference between the new forecast value and the new mean 
            new_var = ((1-(1/n))*old_var) + ((delta*delta2)/n) #Update the value of the sample variance using the population variance formula      
            updated_stats = tf.stack([new_mean, new_var], axis=-1) #Combines the new values of the sample mean and variance 
            current_stats = tf.tensor_scatter_nd_update(current_stats, [[idx]], [updated_stats]) #Overwrites the old values of the mean and variance for a given time series with the new values of the mean and variance 
            return i+1, current_stats, current_count #Set i to i+1 to go to the next time series in the batch, and return the current values of the sample mean and variance, and the number of times the time series have been processed
        i0 = tf.constant(0) #Starting index of the loop 
        stats0 = self.stats #The initial stats for the start of the while loop 
        count0 = self.count #The initial counts that each time series has been processed for the start of the while loop 
        _, updated_stats, updated_count = tf.while_loop( #The while loop which outputs the index, the updated stats, and the updated counts  
            cond=lambda i, *_: i < batch_size, #Determines if the while loop should continue or not, where the loop stops if i exceeds the batch size as there are no more time series to process in the current batch  
            body=loop_body, #Using the function defined earlier which is used in each iteration of the while loop 
            loop_vars=[i0, stats0, count0] #Initial values of the variables in the while loop
        )
        self.stats.assign(updated_stats) #Permanently updates the stats variable of the layer which will be used in future calls of the layer
        self.count.assign(updated_count) #Permanently updates the count variable for the layer which will be used in future calls of the layer 
        out_stats_for_batch = tf.gather(self.stats, series_ids) #Takes the means and variances for the forecasted time points for the time series in the current batch so they can be passed to the proceeding layer 
        return out_stats_for_batch #Returns the means and variances for the forecasted time series in the current batch so they can be passed into the next layer of the network in the current forward pass
class BoltzmannConv1D(Layer): #Defining the Boltzmann convolutional layer (section 4.1) as a subclass of Layer in Keras
    def __init__(self, out_channels, kernel_sizes, dilation_rate=1, padding="causal", temperature=1, **kwargs):#Contructor of the class which takes in several hyperparameters including the type of padding, dilation rate, and temperature of the Boltzmann distribution
        super(BoltzmannConv1D, self).__init__(**kwargs) #Calls the Keras layer class
        self.out_channels = out_channels #Number of output channel
        self.kernel_sizes = kernel_sizes #The set of kernel filter lengths 
        self.dilation_rate = dilation_rate #Dilation rate of the layer
        self.padding = padding #Type of padding for the layer
        self.temperature = temperature #Value of the temperature for the Boltzmann distribution 
        self.convs = [  #Creates a list consisting of sub-convolutional layers with the parameters defined above. The layer is wrapped in a Spatial Concrete Dropout layer 
            ConcreteSpatialDropout1D(
                tf.keras.layers.Conv1D(
                    out_channels, ks,
                    dilation_rate=dilation_rate,
                    padding=padding
                ),
                is_mc_dropout=True,
                weight_regularizer=wr,
                dropout_regularizer=dr
            )
            for ks in kernel_sizes #Iterates over the filter lengths in the kernel_sizes list creating associated convolutional layers
        ]
    def build(self, input_shape): #Create the weights used in the layers
        self.probs = self.add_weight(   #Logits used in the Boltzmann distribution  which are learnable 
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(BoltzmannConv1D, self).build(input_shape) #Finalizes the build methods by inheriting the build from the overall Keras layer class 
    def call(self, x, training=False): #Main method used when calling the Boltzmann layer for a given forward pass
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature) #Defines the probability discussed in section 4.1 
        conv_outputs = [ #Creates the list of the weighted convolution outputs  
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities)) #Turns the kernel probabilities into a list which are then paired with their associated convolutional layer, with them being multiplied together 
        ]
        out = tf.add_n(conv_outputs) #Sums up the weighted convolutional outputs in the list
        return out #Returns the summed output of the weighted convolutional layers according to (2) in section 4.1
class PredefinedBoltzmannConv1D(Layer):
    def __init__(self, filters, kernel_sizes, dilation_rate, custom_filters, temperature=1, **kwargs):
        super(PredefinedBoltzmannConv1D, self).__init__(**kwargs)
        self.filters = filters #Number of filters 
        self.kernel_sizes = kernel_sizes #The set of kernel filter lengths
        self.temperature = temperature #Value of the temperature for the Boltzmann distribution 
        self.custom_filters = custom_filters #Pre-defined filters as described in section 5
        self.dilation_rate = dilation_rate #Dilation rate of the layer 
        self.convs = []
        for ks in kernel_sizes:
            conv_layer = tf.keras.layers.Conv1D(
                filters=filters,
                kernel_size=ks,
                padding="causal",
                use_bias=False,
                dilation_rate=dilation_rate
            )
            self.convs.append(conv_layer) #Creates a list consisting of sub-convolutional layers with hand-crafted filters with different filter lengths
    def build(self, input_shape):
        for i, (conv_layer, ks) in enumerate(zip(self.convs, self.kernel_sizes)): #Create the weights used in the layers 
            conv_layer.build(input_shape)
            filter_weights = self.custom_filters[ks]
            conv_layer.set_weights([filter_weights])
            conv_layer.trainable = False
        self.probs = self.add_weight( #Logits used in the Boltzmann distribution  which are learnable
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(PredefinedBoltzmannConv1D, self).build(input_shape)
    def call(self, x, training=False): #Main method used when calling the hand-crafted Boltzmann layer for a given forward pass
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature)
        conv_outputs = [
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities))
        ]
        out = tf.add_n(conv_outputs)
        return out

In [10]:
kernel_sizes = [3,6,12,18,24,30,36] #Pre-defined filter lengths (section 5)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #Pre-defined filter weights as a function of filter length (4)-(6) in section 5
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #Defining the input time series to ForeClassNet        
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")   #Defining the input time series index to ForeClassNet

#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann Convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connected layer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 5000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#Compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 135, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 135, 3)    │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 135, 128)  │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 135, 3)    │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 135, 128)  │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_30      │ (None, 135, 131)  │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 135, 64)   │  1,081,998 │ concatenate_30[0… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 135, 64)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 135, 32)   │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 135, 32)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 135, 1)    │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 135)       │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      4,353 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 135, 1)    │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 5)         │        166 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 135, 1)    │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,432,451 (5.46 MB)

 Trainable params: 1,377,064 (5.25 MB)

 Non-trainable params: 55,387 (216.36 KB)

In [11]:
#Train ForeClassNet
batch_size = 64 #Batch size used in optimization 
epochs = 100 #Number of epochs used in optimization
tic = time.perf_counter() #Time how long it takes for training
series_id_train = np.arange(Ns, dtype=np.int32) #Create an id for each time series in the test set which will be used in the Welford layer
model.fit(x=[X_observed_train, series_id_train],  y=[X_forecast_train, y_train], batch_size=batch_size,
    epochs=epochs,validation_split=0.1,shuffle=True) #Train ForeClassNet
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #Prints the time it took ForeClassNet to train
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 39s 368ms/step - activation_40_accuracy: 0.4779 - activation_40_loss: 1.4683 - loss: 3.0930 - output_forecast_loss: 2.5912 - output_forecast_mse: 2.5916 - val_activation_40_accuracy: 0.6800 - val_activation_40_loss: 0.8737 - val_loss: 1.0949 - val_output_forecast_loss: 1.2410 - val_output_forecast_mse: 1.2022
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 18s 322ms/step - activation_40_accuracy: 0.7068 - activation_40_loss: 0.7867 - loss: 0.9965 - output_forecast_loss: 1.2063 - output_forecast_mse: 1.2063 - val_activation_40_accuracy: 0.8400 - val_activation_40_loss: 0.5962 - val_loss: 0.5555 - val_output_forecast_loss: 0.9563 - val_output_forecast_mse: 0.9585
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 19s 338ms/step - activation_40_accuracy: 0.8595 - activation_40_loss: 0.5551 - loss: 0.4762 - output_forecast_loss: 0.9486 - output_forecast_mse: 0.9486 - val_activation_40_accuracy: 0.8350 - val_activation_40_loss: 0.5040 - val_loss: 0.3663 - val_output_foreca

In [12]:
def mc_dropout_predict(model, X_test, test_ids, num_samples=100):
    """
    Peforms Monte Carlo dropout, where dropout is applied at inference time. 
    This takes as input the trained ForeClassNet model, the input observed time series, 
    the id's associated with the time series in the test set, and the number of MC samples which
    is the number of forward passes through the trained ForeClassNet model.
    """
    forecast_predictions = [] #List to store the forecast output from each forward pass through ForeClassNet
    classification_predictions = [] #List to store the classification output from each forward pass through ForeClassNet
    for _ in tqdm(range(num_samples), desc="MC Dropout passes"):
        forecast, classification = model.predict([X_test, test_ids], verbose=0) #Performs a forward pass through the network  
        forecast_predictions.append(forecast) #Add the forecast array to the list 
        classification_predictions.append(classification) #Add the classification array of probabilities to the list
    forecast_predictions = np.array(forecast_predictions)
    classification_predictions = np.array(classification_predictions) 
    mean_forecast = np.mean(forecast_predictions, axis=0) #Take the mean with respect to the MC samples of the forecast output
    std_forecast = np.std(forecast_predictions, axis=0) #Take the standard deviation with respect to the MC samples of the forecast output
    mean_classification = np.mean(classification_predictions, axis=0) #Take the mean with respect to the MC samples of the classification probabilities
    std_classification = np.std(classification_predictions, axis=0) #Take the standard deviation with respect to the MC samples of the classification probabilities
    return (mean_forecast, std_forecast), (mean_classification, std_classification)

In [13]:
n_test_series = X_observed_test.shape[0] #Number of time series in the test set 
test_series_ids = np.arange(n_test_series, dtype=np.int32) #Create an id for each time series in the test set for tracking in the Welford mean-variance layer
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #Reset the statistics in the Welford layer

In [14]:
#Perform MC Dropout
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,X_observed_test,    test_series_ids,    num_samples=100) #Perfrom 100 Monte Carlo passes through the trained ForeClassNet
predicted_classes = np.argmax(mean_classification, axis=-1) #Obtain the predicted classes
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy) #Prints the classification accuracy of ForeClassNet on the test set
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes,digits = 3) ##Evaluate the trained ForeClassNet on the test set in terms of classification accuracy and F1 scores
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [03:05<00:00,  1.85s/it]

0.953
              precision    recall  f1-score   support

           0      0.987     1.000     0.993       586
           1      0.932     0.966     0.949       353
           2      0.500     0.611     0.550        18
           3      0.833     0.395     0.536        38
           4      0.000     0.000     0.000         5

    accuracy                          0.953      1000
   macro avg      0.650     0.594     0.605      1000
weighted avg      0.948     0.953     0.947      1000




/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
hc2 = HIVECOTEV2(time_limit_in_minutes= 75) #Initialize the HC2 classifier
y_train_labels = np.argmax(y_train, axis=1) #Convert one hot encoding vector to a single number for each time series
tic = time.perf_counter() #Time how long it takes HC2 to train
hc2.fit(X_observed_train.transpose(0, 2, 1),y_train_labels) #Train the HC2 classifier
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #Prints the time it took the HC2 classifier to train
hc2predictions1 = hc2.predict(X_observed_test.transpose(0, 2, 1)) #Evaluate the trained HC2 classifier on the test set  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), hc2predictions1) #Compute the classification accuracy on the test set
print(f"Classification Test Accuracy: {classification_accuracy}")
from sklearn.metrics import classification_report
classification__report = classification_report(np.argmax(y_test, axis=1),hc2predictions1,digits = 3) #Evaluate the trained HC2 classifier on the test set in terms of classification accuracy and F1 scores
print(classification__report)

Time taken for fitting: 3733.87 seconds
Classification Test Accuracy: 0.962
              precision    recall  f1-score   support

           0      0.993     1.000     0.997       586
           1      0.930     0.983     0.956       353
           2      0.667     0.667     0.667        18
           3      0.895     0.447     0.596        38
           4      0.000     0.000     0.000         5

    accuracy                          0.962      1000
   macro avg      0.697     0.619     0.643      1000
weighted avg      0.956     0.962     0.956      1000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
a = b = c = d = 0 #Initialize variables to store counts for the contingency table used in McNemar's test
for i in range(len(y_test)): #Loop through the number of examples in the test set 
    correct_1 = (MRpredictions[i] == np.argmax(y_test[i])) #Returns 1 if MR-Hydra gives a correct prediction and 0 otherwise
    correct_2 = (predicted_classes[i] == np.argmax(y_test[i])) #Returns 1 if ForeClassNet gives a correct prediction and 0 otherwise
    if  correct_1 and  correct_2:
        a += 1 #Both classifiers gave the correct prediction
    elif correct_1 and not correct_2:
        b += 1 #MR-Hydra gave the correct prediction and ForeClassNet did not
    elif not correct_1 and correct_2:
        c += 1 #ForeClassNet gave the correct prediction and MR-Hydra did not
    else:
        d += 1 #Both classifiers gave the incorrect prediction 
table = [[a, b],
         [c, d]] #Construct contingency table 
print("Contingency table:")
print(np.array(table))
result = mcnemar(table, exact=False, correction=True) #Perform McNemar's test 
print("statistic:", result.statistic) #Return the test statistic from McNemar's test
print("p-value:  ", result.pvalue) #Return the p-value from McNemar's test

Contingency table:
[[948  10]
 [  5  37]]
statistic: 1.0666666666666667
p-value:   0.30169958247834494


In [17]:
a = b = c = d = 0 #Initialize variables to store counts for the contingency table used in McNemar's test
for i in range(len(y_test)): #Loop through the number of examples in the test set 
    correct_1 = (hc2predictions1[i] == np.argmax(y_test[i])) #Returns 1 if HC2 gives a correct prediction and 0 otherwise
    correct_2 = (predicted_classes[i] == np.argmax(y_test[i])) #Returns 1 if ForeClassNet gives a correct prediction and 0 otherwise
    if  correct_1 and  correct_2:
        a += 1 #Both classifiers gave the correct prediction
    elif correct_1 and not correct_2:
        b += 1 #HC2 gave the correct prediction and ForeClassNet did not 
    elif not correct_1 and correct_2:
        c += 1 #ForeClassNet gave the correct prediction and HC2 did not
    else:
        d += 1 #Both classifiers gave the incorrect prediction 
table = [[a, b],
         [c, d]] #Construct contingency table 
print("Contingency table:")
print(np.array(table))
result = mcnemar(table, exact=False, correction=True) #Perform McNemar's test
print("statistic:", result.statistic) #Return the test statistic from McNemar's test
print("p-value:  ", result.pvalue) #Return the p-value from McNemar's test

Contingency table:
[[950  12]
 [  3  35]]
statistic: 4.266666666666667
p-value:   0.03886710381241731


In [18]:
a = b = c = d = 0 #Initialize variables to store counts for the contingency table used in McNemar's test 
for i in range(len(y_test)): #Loop through the number of examples in the test set 
    correct_1 = (clf_predictions1[i] == np.argmax(y_test[i])) #Returns 1 if Inception time gives a correct prediction and 0 otherwise
    correct_2 = (predicted_classes[i] == np.argmax(y_test[i])) #Returns 1 if ForeClassNet gives a correct prediction and 0 otherwise
    if  correct_1 and  correct_2:
        a += 1 #Both classifiers gave the correct prediction
    elif correct_1 and not correct_2:
        b += 1 #Inception time gave the correct prediction and ForeClassNet did not 
    elif not correct_1 and correct_2:
        c += 1 #ForeClassNet gave the correct prediction and Inception time did not
    else:
        d += 1 #Both classifiers gave the incorrect prediction
table = [[a, b],
         [c, d]] #Construct contingency table 
print("Contingency table:")
print(np.array(table))
result = mcnemar(table, exact=False, correction=True) #Perform McNemar's test 
print("statistic:", result.statistic) #Return the test statistic from McNemar's test
print("p-value:  ", result.pvalue) #Return the p-value from McNemar's test

Contingency table:
[[943  13]
 [ 10  34]]
statistic: 0.17391304347826086
p-value:   0.6766573217164242


In [19]:
a = b = c = d = 0 #Initialize variables to store counts for the contingency table used in McNemar's test 
for i in range(len(y_test)): #Loop through the number of examples in the test set 
    correct_1 = (Teaserpredictions[i] == np.argmax(y_test[i])) #Returns 1 if TEASER gives a correct prediction and 0 otherwise
    correct_2 = (predicted_classes[i] == np.argmax(y_test[i])) #Returns 1 if ForeClassNet gives a correct prediction and 0 otherwise
    if  correct_1 and  correct_2:
        a += 1 #Both classifiers gave the correct prediction
    elif correct_1 and not correct_2:
        b += 1 #TEASER gave the correct prediction and ForeClassNet did not
    elif not correct_1 and correct_2:
        c += 1 #ForeClassNet gave the correct prediction and TEASER did not
    else:
        d += 1 #Both classifiers gave the incorrect prediction 
table = [[a, b],
         [c, d]] #Construct contingency table 
print("Contingency table:")
print(np.array(table))
result = mcnemar(table, exact=False, correction=True) #Perform McNemar's test
print("statistic:", result.statistic) #Return the test statistic from McNemar's test
print("p-value:  ", result.pvalue) #Return the p-value from McNemar's test

Contingency table:
[[946  10]
 [  7  37]]
statistic: 0.23529411764705882
p-value:   0.6276258050283593
